In [1]:
!pip install julia

In [2]:
import julia
julia.install()
from julia import Base


In [4]:
#from https://gist.github.com/szagoruyko/dccce13465df1542621b728fcc15df53
import torch
from cupy.cuda import function
from pynvrtc.compiler import Program
from collections import namedtuple

a = torch.randn(1,4,4).cuda()
b = torch.zeros(a.size()).cuda()

kernel = '''
extern "C"
__global__ void flip(float *dst, const float *src, int w, int total)
{
   int i = blockIdx.x * blockDim.x + threadIdx.x;
   if(i >= total)
      return;
   dst[i] = src[(i / w) * w + (w - (i % w) - 1)];
}
'''
program = Program(kernel, 'flip.cu')
ptx = program.compile()

m = function.Module()
m.load(bytes(ptx.encode()))

f = m.get_function('flip')

Stream = namedtuple('Stream', ['ptr'])
s = Stream(ptr=torch.cuda.current_stream().cuda_stream)

f(grid=(1,1,1), block=(1024,1,1), args=[b.data_ptr(), a.data_ptr(), a.size(-1), a.numel()],
  stream=s)

print(a)
print(b)


ImportError: 
================================================================
Failed to import CuPy.

If you installed CuPy via wheels (cupy-cudaXXX or cupy-rocm-X-X), make sure that the package matches with the version of CUDA or ROCm installed.

On Linux, you may need to set LD_LIBRARY_PATH environment variable depending on how you installed CUDA/ROCm.
On Windows, try setting CUDA_PATH environment variable.

Check the Installation Guide for details:
  https://docs.cupy.dev/en/latest/install.html

Original error:
  ImportError: cannot import name _util
================================================================


In [ ]:
from julia import Main
Main.eval("""
using CUDA
function aa(arr)
return sum(CuArray(arr))
end

""")

In [ ]:
from julia import Main
import numpy as np
Main.aa(np.array([2,2,2]))

In [ ]:
from julia import Main
import numpy as np
arr= np.array([2,2,2])
sum(CUDA.ones(2,2,2))
""")